In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
# from edstan_data import EdstanData
from frvrs_utils import FRVRSUtilities
from notebook_utils import NotebookUtilities
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp
import pandas as pd
import re

nu = NotebookUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)
fu = FRVRSUtilities(
    data_folder_path=osp.abspath('../data'),
    saves_folder_path=osp.abspath('../saves')
)
from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(
    s=nu,
    secrets_json_path=os.path.abspath(os.path.join(nu.data_folder, 'secrets', 'bbai_secrets.json'))
)


# Parse Domain Documents for Entities

Downloaded all documents from https://nextcentury.atlassian.net/wiki/spaces/ITMC/pages/2991849482/Domain+Documents and converted them all to PDF files and stored them in the data folder.

In [3]:

import logging

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


## Option 1: Use a Hugging Face NER model

In [4]:

from llama_index import postprocessor

dir(postprocessor)

['AutoPrevNextNodePostprocessor', 'CohereRerank', 'EmbeddingRecencyPostprocessor', 'FixedRecencyPostprocessor', 'KeywordNodePostprocessor', 'LLMRerank', 'LongContextReorder', 'LongLLMLinguaPostprocessor', 'MetadataReplacementPostProcessor', 'NERPIINodePostprocessor', 'PIINodePostprocessor', 'PrevNextNodePostprocessor', 'SentenceEmbeddingOptimizer', 'SentenceTransformerRerank', 'SimilarityPostprocessor', 'TimeWeightedPostprocessor', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'cohere_rerank', 'llm_rerank', 'longllmlingua', 'metadata_replacement', 'node', 'node_recency', 'optimizer', 'pii', 'sbert_rerank', 'types']

In [5]:

from llama_index import ServiceContext
from llama_index.postprocessor import NERPIINodePostprocessor

# Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization
os.environ['OPENAI_API_KEY'] = wsu.secrets_json['OPENAI_API_KEY']
service_context = ServiceContext.from_defaults()
processor = NERPIINodePostprocessor(service_context=service_context)

In [14]:

# load documents
import PyPDF2
from llama_index.schema import TextNode

pdf_folder = '../data/Domain_Knowledge'
with open(osp.join(pdf_folder, 'DoDTR-Data-Dictionary-External.pdf'), 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    text = ''
    for page_number in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_number]
        text += page.extract_text()
node = TextNode(text=text)

In [ ]:

from llama_index.postprocessor import (
    PIINodePostprocessor,
    NERPIINodePostprocessor,
)
from llama_index.llms import HuggingFaceLLM
from llama_index import ServiceContext, Document, VectorStoreIndex

In [16]:

from llama_index.schema import NodeWithScore

new_nodes = processor.postprocess_nodes([NodeWithScore(node=node)])

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [17]:

# Get mapping in metadata
new_nodes[0].node.metadata["__pii_node_info__"]

{'[ORG_3]': 'DOD TRAUM', '[ORG_124]': 'Joint T', '[MISC_131]': '##ra', '[ORG_133]': '##uma System', '[ORG_145]': 'JTS', '[ORG_157]': 'Department of Defense', '[ORG_180]': 'DoD', '[MISC_253]': 'Joint Trauma System', '[MISC_297]': 'Sailor', '[MISC_305]': 'Marine', '[MISC_316]': 'Air', '[ORG_478]': 'JTS', '[ORG_654]': 'JTS', '[MISC_751]': 'M', '[ORG_799]': 'DoD T', '[MISC_804]': '##rauma', '[ORG_810]': 'Registry', '[ORG_821]': 'DoDTR', '[ORG_936]': 'DoD', '[ORG_1109]': 'DoDTR', '[MISC_1151]': 'Global War on Terror', '[MISC_1174]': 'G', '[MISC_1177]': '##T', '[ORG_1335]': 'Combat', '[MISC_1342]': 'Trauma', '[ORG_1349]': 'Registry', '[ORG_1359]': 'CTR', '[ORG_1397]': 'Center for AMEDD Strategic Studies', '[ORG_1433]': 'CASS', '[LOC_1579]': 'Landstuhl Regional Medical Center', '[ORG_1615]': 'LRMC', '[LOC_1624]': 'Germany', '[ORG_1687]': 'Army', '[ORG_1789]': 'Joint', '[ORG_1804]': '##ra', '[ORG_1810]': 'System', '[ORG_1818]': 'J', '[ORG_1821]': '##S', '[ORG_1846]': 'Joint', '[ORG_1853]': '##


## Option 2: Use SpaCy

In [26]:

import spacy

# Example usage
text = 'Barack Obama was the 44th President of the United States.'

# Load the spaCy model
try: nlp = spacy.load('en_core_web_sm')
except OSError as e:
    print(str(e).strip())
    command_str = f'{sys.executable} -m spacy download en_core_web_sm --quiet'
    print(command_str)
    !{command_str}
    nlp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()

# Process the text
doc = nlp(text)

# Extract named entities
entities = []
for ent in doc.ents:
    # for fn in dir(ent):
    #     value = eval(f'ent.{fn}')
    #     print(f'{fn}: {value}')
    entities.append({
        'type': ent.label_,
        'text': ent.text, # Or lemma_, orth_, text_with_ws
        'start_pos': ent.start_char, # Or start
        'end_pos': ent.end_char # Or end
    })
print(entities)

[{'type': 'PERSON', 'text': 'Barack Obama', 'start_pos': 0, 'end_pos': 12}, {'type': 'ORDINAL', 'text': '44th', 'start_pos': 21, 'end_pos': 25}, {'type': 'GPE', 'text': 'the United States', 'start_pos': 39, 'end_pos': 56}]
